In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPool2D, Conv3D, MaxPool3D, UpSampling2D, GlobalMaxPool2D, GlobalMaxPool3D, GlobalAveragePooling2D, GlobalAveragePooling3D, Conv2DTranspose, concatenate
from tensorflow.keras.layers import Dense, Dropout, Activation, Reshape, Flatten, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical, plot_model

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import NASNetMobile, Xception, DenseNet121, MobileNetV2, InceptionV3, InceptionResNetV2, vgg16, resnet50, inception_v3, xception, DenseNet201

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import jaccard_score
from sklearn.cluster import KMeans

from scipy import stats

import seaborn as sns

import skimage
from skimage.transform import rotate

from tqdm.notebook import tqdm
from datetime import datetime

from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_auc_score, roc_curve
import numpy as np
import os
import cv2
import pandas as pd
# import imutils
import random
from PIL import Image
import matplotlib.pyplot as plt

import pickle
import urllib

import warnings
warnings.filterwarnings("ignore")

from PIL import Image

import tensorflow_addons as tfa

from IPython.display import HTML

import pydicom
from pydicom.pixel_data_handlers.util import apply_color_lut
import re

import itertools

from sklearn.utils import shuffle


# Define

In [ ]:
im_size = (256,256)
num_im = 64
batch_size = 48
split_size = (0.7,0.3,0.0)
random_state = 42
image_threshold = 0.5
test_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'

# Load Images

In [ ]:
def index_train_valid_test_split(data, is_shuffle=True, split_size=split_size):
    
    length = len(data)
    train_end = int(split_size[0] * length)
    
    valid_start = int(train_end)
    valid_end = int((split_size[0] + split_size[1]) * length)
    
    test_start = int(valid_end)
    
    if is_shuffle:
        data = shuffle(data)
        
    return data[:train_end], data[valid_start:valid_end], data[test_start:]

In [ ]:
df_test = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
df_test['BraTS21ID'] = df_test['BraTS21ID'].apply(lambda x: f'{"0"*(5-len(str(x)))}{x}')
df_test

In [ ]:
def preprocess(im, im_size=im_size):
    im = cv2.resize(im, dsize=im_size)
    return im

def scale_image(im):
    max_im = np.max(im)
    min_im = np.min(im)
    
    if max_im == min_im:
        return im - min_im
    
    return (im - min_im) / (max_im - min_im)

def get_image(path, im_size=im_size):
    ds = pydicom.dcmread(path)

    im = ds.pixel_array

    im = scale_image(im)
    
    im = preprocess(im, im_size)
    
    return np.array(im)

In [ ]:
def get_id_image(scan_id, types, path_dir=test_dir, im_size=im_size, num_im=num_im, black_threshold=0.8, is_tqdm=True):
    files = sorted(os.listdir(f'{path_dir}/{scan_id}/{types}'),
               key=lambda x: int(''.join(re.findall(r'[0-9]+', x))))
    
    im_list = []
    
    step = len(files) / num_im
    
    files = [files[int(i*step)] for i in range(num_im)]
    
    if is_tqdm:
        files = tqdm(files)
        
    for file in files:
        file_path = f'{path_dir}/{scan_id}/{types}/{file}'
        
        im = get_image(file_path, im_size=im_size)
            
        im_list += [im]
        
    im_list = np.moveaxis(np.array(im_list), 0, -1)
         
    im_list = im_list.reshape(*im_list.shape, 1)
    
    return tf.image.convert_image_dtype(im_list, tf.float32)

# Model

In [ ]:
with tf.device('/device:GPU:0'):

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# with tpu_strategy.scope():
    def get_model():
        inputs = Input(shape=(*im_size, num_im, 1))
    
        x = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
        x = MaxPool3D(pool_size=2)(x)
        x = BatchNormalization()(x)

        x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
        x = MaxPool3D(pool_size=2)(x)
        x = BatchNormalization()(x)

        x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
        x = MaxPool3D(pool_size=2)(x)
        x = BatchNormalization()(x)

        x = Conv3D(filters=256, kernel_size=3, activation="relu")(x)
        x = MaxPool3D(pool_size=2)(x)
        x = BatchNormalization()(x)

        x = GlobalAveragePooling3D()(x)
        x = Dense(units=512, activation="relu")(x)
        x = Dropout(0.3)(x)

        outputs = Dense(units=1, activation="sigmoid")(x)

        model = Model(inputs=inputs, outputs=outputs, name='3dcnn')
        model.compile(loss='binary_crossentropy',
                      optimizer='adam', 
                      metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

        return model

    get_model().summary()

# Dataset

In [ ]:
from tensorflow.keras.utils import Sequence
import math
class Dataset(Sequence):
    def __init__(self, df, types, is_train=True, batch_size=3, shuffle=True):
        self.paths = df['BraTS21ID'].values
        self.y =  df['MGMT_value'].values
        self.is_train = is_train
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.types = types

    def __len__(self):
        return math.ceil(len(self.paths)/self.batch_size)

    def __getitem__(self, ids):

        batch_paths = self.paths[ids * self.batch_size:(ids + 1) * self.batch_size]

        if self.y is not None:
            batch_y = self.y[ids * self.batch_size: (ids + 1) * self.batch_size]

        list_x =  [get_id_image(id_path, self.types, is_tqdm=False) for id_path in batch_paths]

        batch_X = np.stack(list_x)

        if self.is_train:
            return batch_X, batch_y
        else:
            return batch_X

    def on_epoch_end(self):
        if self.shuffle and self.is_train:
            ids_y = list(zip(self.paths, self.y))
            shuffle(ids_y)
            self.paths, self.y = list(zip(*ids_y))

# Predict Model

In [ ]:
# with tpu_strategy.scope():
with tf.device('/device:GPU:0'):
    model_FLAIR = get_model()
    model_FLAIR.load_weights('../input/3d-brian-tumor-weight/best_3d_FLAIR_model_numim_64_AUC.h5')
    
    test_dataset_FLAIR = Dataset(df_test, 'FLAIR', is_train=False)

    FLAIR_pred = model_FLAIR.predict(test_dataset_FLAIR)[:, 0]

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(FLAIR_pred);

In [ ]:
# with tpu_strategy.scope():
with tf.device('/device:GPU:0'):
    model_T1wCE = get_model()
    model_T1wCE.load_weights('../input/3d-brian-tumor-weight/best_3d_T1wCE_model_numim_64_AUC.h5')
    
    test_dataset_T1wCE = Dataset(df_test, 'T1wCE', is_train=False)

    T1wCE_pred = model_T1wCE.predict(test_dataset_T1wCE)[:, 0]

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(T1wCE_pred);

In [ ]:
mean_prediction = np.mean([FLAIR_pred, T1wCE_pred], axis=0)

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(mean_prediction);

In [ ]:
submission = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
submission['MGMT_value'] = T1wCE_pred
submission

In [ ]:
submission.to_csv('submission.csv', index=False)